In [ ]:
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin

# The website you want to scrape
url = "https://pokemondb.net/sprites"  

# Folder to save images
os.makedirs("national", exist_ok=True)

# Get the HTML content of the page
headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# Find all <img> tags
img_tags = soup.find_all("img")

# Download each image
for i, img in enumerate(img_tags):
    img_url = img.get("src")

    # Skip if no src found
    if not img_url:
        continue

    # Convert relative URLs to absolute
    full_url = urljoin(url, img_url)

    # Extract image filename
    filename = os.path.join("national", f"image_{i}.jpg")  
    
    try:
        img_data = requests.get(full_url).content
        with open(filename, "wb") as f:
            f.write(img_data)
        print(f"✅ Saved: {filename}")
    except Exception as e:
        print(f"❌ Failed to download {full_url}: {e}")

In [11]:
import requests
from bs4 import BeautifulSoup
import os
import re

# Starting URL
base_url = "https://archives.bulbagarden.net"
start_url = "/wiki/Category:Type_icons"

# Folder to save images
save_folder = "type_icons"
os.makedirs(save_folder, exist_ok=True)

current_url = start_url

while current_url:
    # Request page
    print(f"Scraping page: {base_url + current_url}")
    response = requests.get(base_url + current_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all image links ending with 'Sleep.png'
    for link in soup.find_all('a', href=True):
        href = link['href']
        if href.endswith("Sleep.png"):
            file_page_url = base_url + href

            # Visit the file page
            file_page = requests.get(file_page_url)
            file_soup = BeautifulSoup(file_page.text, 'html.parser')

            # Find the actual .png URL
            img = file_soup.find('a', href=re.compile(r'\.png$'))
            if img:
                img_url = img['href']
                if img_url.startswith("//"):
                    img_url = "https:" + img_url

                print(f"Downloading {img_url}")

                # Download the image
                img_data = requests.get(img_url).content
                img_name = img_url.split('/')[-1]
                with open(os.path.join(save_folder, img_name), 'wb') as handler:
                    handler.write(img_data)

    # Find link to the next page
    next_link = soup.find('a', text="next page")
    if next_link:
        current_url = next_link['href']
    else:
        current_url = None  # No more pages


Scraping page: https://archives.bulbagarden.net/wiki/Category:Type_icons


C:\Users\tisor\AppData\Local\Temp\ipykernel_28824\2525020903.py:48: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  next_link = soup.find('a', text="next page")


Scraping page: https://archives.bulbagarden.net/w/index.php?title=Category:Type_icons&filefrom=FireIC+XD.png#mw-category-media
Scraping page: https://archives.bulbagarden.net/w/index.php?title=Category:Type_icons&filefrom=Normal+icon+PE.png#mw-category-media
Scraping page: https://archives.bulbagarden.net/w/index.php?title=Category:Type_icons&filefrom=Tretta+Fighting+type.png#mw-category-media
